In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# import matplotlib to show plots inline.
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import glob
import os

import astropy.units as u
import matplotlib as mpl
import sys 
sys.path.append("..")
from plot_OpSims import get_metric_medians
from script_utils import get_opsim_areas

In [3]:
# import maf python modules
import lsst.sims.maf.db as db
import lsst.sims.maf.metrics as metrics
import lsst.sims.maf.slicers as slicers
import lsst.sims.maf.stackers as stackers
import lsst.sims.maf.plots as plots
import lsst.sims.maf.metricBundles as metricBundles

In [4]:
# import convenience functions
import sys 
sys.path.append("../../LSST_OpSim")
from Scripts_NBs.opsimUtils import *

## Load the results
We first load the results of the run into memory. We will read jointly the FBS v1.5 and FBS v1.6 results, and will differentiate at the time of plotting.

In [5]:
NSIDE = 64
your_username = "rjassef"
folder_mafoutput = "EM5_depths_{0:d}".format(NSIDE)
#folder_mafoutput = "Lstar_dust"
resultDbPath = '/home/idies/workspace/Storage/{0}/persistent/MAFOutput/{1}'.format(
    your_username, folder_mafoutput)
metricDataPath = '/home/idies/workspace/Storage/{0}/persistent/MAFOutput/{1}/MetricData/'.format(
    your_username, folder_mafoutput)

In [6]:
# get a dictionary of resultDb from given directory
resultDbs = getResultsDbs(resultDbPath)

In [7]:
# retrieve metricBundles for each opsim run and store them in a dictionary
bundleDicts_raw = dict()
for runName in resultDbs:
    bundleDicts_raw[runName] = bundleDictFromDisk(resultDbs[runName], runName, metricDataPath)

Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Healpix slicer using NSID

In [8]:
#Rearrange the bundleDicts_raw dictionary so that the keys are always the same as for the first run.
dbRuns = list(bundleDicts_raw.keys())
Keys = list(bundleDicts_raw[dbRuns[0]].keys())
bundleDicts = dict()
for runName in dbRuns:
    bundleDicts[runName] = dict()
    Keys_raw = bundleDicts_raw[runName]
    for Key in Keys:
        for Key_raw in Keys_raw:
            if Key[1]==Key_raw[1]:
                bundleDicts[runName][Key] = bundleDicts_raw[runName][Key_raw]

In [9]:
# check keys
dbRuns = list(resultDbs.keys())
bd_keys = list(bundleDicts[dbRuns[0]].keys())
print(bd_keys)

[(1, 'ExgalM5_with_cuts_AGN_g'), (2, 'ExgalM5_with_cuts_AGN_u')]


In [10]:
uKey, gKey = (2, 'ExgalM5_with_cuts_AGN_u'), (1, 'ExgalM5_with_cuts_AGN_g')

## Process the metric results for each OpSim and save the results in a table.

Note that in general, the rank will be from worst (0%) to best (100%)

#### u-band, 3-sigma depth

In [11]:
def mlim_3sigma(mlim_5sigma):
    return mlim_5sigma + 2.5*np.log10(5./3.)

In [12]:
u_mds = get_metric_medians(uKey, bundleDicts, data_func=mlim_3sigma)
u_rank_sort = np.argsort(u_mds)
print(u_mds[u_rank_sort[0]], u_mds[u_rank_sort[-1]])

25.63183128525153 26.64337935736984


#### g-band, 5-sigma depth

In [13]:
g_mds = get_metric_medians(gKey, bundleDicts)
g_rank_sort = np.argsort(g_mds)

#### Get the sky areas covered by each OpSim run

In [14]:
u_areas = get_opsim_areas(uKey, bundleDicts)
g_areas = get_opsim_areas(gKey, bundleDicts)
print(u_areas[22], g_areas[22])

21965.153988821447 25559.848671260872


#### Save the table

In [15]:
fname = "Table_ug_band_depth.txt"
tab_file = open(fname,"w")
for k,run in enumerate(list(bundleDicts.keys())):
    tab_file.write("{0:50s}".format(run))
    
    u_rank_percent = (np.argwhere(k==u_rank_sort)[0,0]+1)/len(u_mds) * 100.
    tab_file.write(" {0:10.2f} {1:5.1f}".format(u_mds[k], u_rank_percent))
    tab_file.write(" {0:10.0f}".format(u_areas[k]))
               
    g_rank_percent = (np.argwhere(k==g_rank_sort)[0,0]+1)/len(g_mds) * 100.
    tab_file.write(" {0:10.2f} {1:5.1f}".format(g_mds[k], g_rank_percent))
    tab_file.write(" {0:10.0f}".format(g_areas[k]))
                   
    tab_file.write("\n")
tab_file.close()